In [ ]:
!pip install azure-storage-blob

In [ ]:
# Import libraries
from io import StringIO
import pandas as pd
import numpy as np
import json
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [ ]:
# Extracting from datasource
# The URL from which to download the CSV
URL = "https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?fourfour=h9gi-nx95&cacheBust=1715640490&date=20240514&accessType=DOWNLOAD"

# Send a GET request to the URL
response = requests.get(URL)

# Check if the request was successful
if response.status_code == 200:
    # Open a file in binary write mode
    with open('vehicle_collisions.csv', 'wb') as file:
        # Write the content of the response to the file
        file.write(response.content)
    print("CSV file has been downloaded and saved as 'vehicle_collisions.csv'.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

CSV file has been downloaded and saved as 'vehicle_collisions.csv'.


In [ ]:
# Load the CSV file into a DataFrame
df_raw = pd.read_csv('vehicle_collisions.csv')
print(df_raw.info())
print(df_raw.shape)
df_raw.head()

<ipython-input-7-d2bf80e979a5>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('vehicle_collisions.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2087324 entries, 0 to 2087323
Data columns (total 29 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   CRASH DATE                     object 
 1   CRASH TIME                     object 
 2   BOROUGH                        object 
 3   ZIP CODE                       object 
 4   LATITUDE                       float64
 5   LONGITUDE                      float64
 6   LOCATION                       object 
 7   ON STREET NAME                 object 
 8   CROSS STREET NAME              object 
 9   OFF STREET NAME                object 
 10  NUMBER OF PERSONS INJURED      float64
 11  NUMBER OF PERSONS KILLED       float64
 12  NUMBER OF PEDESTRIANS INJURED  int64  
 13  NUMBER OF PEDESTRIANS KILLED   int64  
 14  NUMBER OF CYCLIST INJURED      int64  
 15  NUMBER OF CYCLIST KILLED       int64  
 16  NUMBER OF MOTORIST INJURED     int64  
 17  NUMBER OF MOTORIST KILLED      int64  
 18  CO

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
  config = json.load(config_file)

CONNECTION_STRING_AZURE_STORAGE = "DefaultEndpointsProtocol=https;AccountName=cis4400spring2024mohina;AccountKey=IxrJzjP2LoIb4fbZPu5ge2Ao1+ZnGPz+XGukf0sPBs50gcC50LY5RSjgaKky2KcDsPNk5+Y/O/pR+AStvbbwjA==;EndpointSuffix=core.windows.net"
CONTAINER_AZURE = 'vehiclecollisions'
blob_name = "vehicle_collisions.csv"

# Convert DataFrame to CSV
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded vehicle_collisions.csv to Azure Blob Storage in container vehiclecollisions.
